## Import Lib
note: pyWinhook only available for window

In [1]:
import pyWinhook
import pythoncom
import os
import matplotlib.pyplot as plt
import json
import numpy as np
import sys
import pandas as pd
from IPython.display import clear_output
import csv
from scipy.spatial.distance import cityblock
from sklearn.metrics import roc_curve
from sklearn.cluster import KMeans

### init variable and function

In [2]:
global userName
userFilePath = "Collecting_keyStorke.csv"

In [3]:
class KeyLogger:
    def __init__(self,register = True):
        self.enterPressed = False
        self.eventList = []
        self.isCaps = False
        self.register = register
        #self.message = ""
        
    def keyDownEvent(self, event):
        if event.KeyID == 20 and self.isCaps == False:
            self.isCaps = True
        elif event.KeyID == 20 and self.isCaps == True:
            self.isCaps = False     
        if event.KeyID>= 48 and event.KeyID<=57:
            event.Ascii = event.KeyID
        if self.isCaps == True and event.Ascii>=97 and event.Ascii<=122:
            event.Ascii = event.KeyID
        self.storeEvent("Down", event) 
        return True
    
    def mainLoop(self):
        while not self.enterPressed:
            pythoncom.PumpWaitingMessages()

    def storeEvent(self, activity, event):
        keystrokeTime = int(event.Time)
        self.eventList.append ((userName,event.Ascii,activity, int(keystrokeTime)))

        # Chosen to use Escape key (ESC) due to input using a similar method
        # Enter Key - KeyCode: 13 Ascii: 13 ScanCode: 28 - ESC = 27 @ Ascii
        if event.Ascii == 27:
            self.enterPressed = True
            if self.register:
                userRecordData(self.eventList)
            
def userRecordData(eventList):
    print("\noutput")
    print(eventList)
    with open(userFilePath,'a',newline='\n') as f:
        writer = csv.writer(f)
        #writer.writerow(['user','key','keyEvent','Time'])
        writer.writerows(eventList)
    f.close()   
    
def getUserName():
    global userName
    userName = input("Enter your Name: ")

def getKeyStroke(register = True):
    
    keyLogger = KeyLogger(register = register)
    hookManager = pyWinhook.HookManager()
    hookManager.KeyDown = keyLogger.keyDownEvent
    hookManager.HookKeyboard()

    keyLogger.mainLoop()
    # Unhooks the keyboard, no more data recorded, returns to menu
    hookManager.UnhookKeyboard()
    
    return keyLogger
    


## Add new user

In [13]:
getUserName()
print("Enter your text: ")
print("type the following word: ")
print("fascinated interesting madly miniature flesh rain")
getKeyStroke()

Enter your Name: tester2
Enter your text: 
type the following word: 
fascinated interesting madly miniature flesh rain

output
[('tester2', 102, 'Down', 622682468), ('tester2', 97, 'Down', 622684375), ('tester2', 115, 'Down', 622685828), ('tester2', 99, 'Down', 622688078), ('tester2', 105, 'Down', 622689937), ('tester2', 110, 'Down', 622691375), ('tester2', 97, 'Down', 622693265), ('tester2', 116, 'Down', 622694625), ('tester2', 101, 'Down', 622696218), ('tester2', 100, 'Down', 622697562), ('tester2', 32, 'Down', 622699234), ('tester2', 105, 'Down', 622700546), ('tester2', 110, 'Down', 622702500), ('tester2', 116, 'Down', 622703796), ('tester2', 101, 'Down', 622705437), ('tester2', 114, 'Down', 622706703), ('tester2', 101, 'Down', 622707953), ('tester2', 115, 'Down', 622709171), ('tester2', 116, 'Down', 622710671), ('tester2', 105, 'Down', 622712140), ('tester2', 110, 'Down', 622713218), ('tester2', 103, 'Down', 622714484), ('tester2', 32, 'Down', 622715734), ('tester2', 109, 'Down', 6

getUserName()
print("Enter your text: ")
getKeyStroke()

### Process typing stroke

In [4]:
data = pd.read_csv("Collecting_keyStorke.csv")
userList = data.user.unique()
keyList = data.key.unique()
df = pd.DataFrame(columns=['subject','key','time'])
for i in range(0,len(userList)):
    queryData = data.query("user=='" +userList[i]+"'")
    queryLen = len(queryData)
    finalData = {}
    for j in range(0,queryLen-1):
        finalData['subject'] = userList[i] # get user
        finalData['key'] = chr(queryData.iloc[j].key) + " " + chr(queryData.iloc[j+1].key)
        finalData['time'] = (int(queryData.iloc[j+1].Time) - int(queryData.iloc[j].Time))/1000
        df = df.append(finalData,ignore_index=True )
                
f = open("KeyStrokeDistance.csv", 'w',newline='\n')
writer = csv.writer(f)
writer.writerow(['subject','key','time'])
for row in df.iterrows():
    #print(row[1])
    writer.writerow(row[1])
    
f.close()
print(df[df.subject == 'tester'])

   subject  key   time
0   tester  f a  0.110
1   tester  a s  0.094
2   tester  s c  0.640
3   tester  c i  0.110
4   tester  i n  0.265
5   tester  n a  0.094
6   tester  a t  0.219
7   tester  t e  0.109
8   tester  e d  0.172
9   tester  d    0.391
10  tester    i  0.453
11  tester  i n  0.172
12  tester  n t  0.140
13  tester  t e  0.125
14  tester  e r  0.078
15  tester  r e  2.188
16  tester  e s  0.094
17  tester  s t  1.125
18  tester  t i  0.281
19  tester  i n  0.156
20  tester  n g  0.094
21  tester  g    0.515
22  tester    m  0.407
23  tester  m a  0.093
24  tester  a d  0.110
25  tester  d l  0.109
26  tester  l y  0.250
27  tester  y    0.625
28  tester    m  0.469
29  tester  m i  0.172
30  tester  i n  0.156
31  tester  n i  0.172
32  tester  i a  0.797
33  tester  a t  0.312
34  tester  t u  0.125
35  tester  u r  0.094
36  tester  r e  0.094
37  tester  e    0.234
38  tester    f  0.735
39  tester  f l  0.093
40  tester  l e  0.094
41  tester  e s  0.094
42  tester 

In [5]:
df_avg = pd.DataFrame(columns=['subject','key','time'])
for user in df.subject.unique():
    temp = df[df.subject == user].groupby(['key']).mean()
    temp.insert(0,'subject',[user]*temp.shape[0])
    df_avg = df_avg.append(temp.reset_index())
df_avg

,subject,key,time
0,tester,f,0.735
1,tester,i,0.453
2,tester,m,0.438
3,tester,r,0.656
4,tester,a d,0.110
...,...,...,...
35,tester2,t e,1.617
36,tester2,t i,1.469
37,tester2,t u,2.453
38,tester2,u r,1.234


In [6]:
df_pivot = df_avg.pivot(index='subject', columns='key', values = 'time')
df_pivot

key,f,i,m,r,a d,a i,a s,a t,c i,d,...,r a,r e,s c,s h,s t,t e,t i,t u,u r,y
subject,,,,,,,,,,,,,,,,,,,,,
tester,0.735,0.453,0.4380,0.656,0.110,0.094,0.094,0.2655,0.110,0.391,...,0.094,1.1410,0.64,0.156,1.125,0.117,0.281,0.125,0.094,0.625
tester2,1.360,1.312,0.9295,1.063,1.375,1.110,1.453,1.2425,1.859,1.672,...,1.109,1.1405,2.25,1.140,1.500,1.617,1.469,2.453,1.234,1.125


### Train KNeighbors

In [7]:
from sklearn.neighbors import KNeighborsClassifier
clf_df = df_pivot
clf = KNeighborsClassifier(n_neighbors=1)
clf.fit(clf_df[clf_df.columns[1:]].to_numpy(), userList)

KNeighborsClassifier(n_neighbors=1)

## Verify user

In [8]:
#getUserName()
global userName
userName = 'guest'
print("Enter your text: ")
print("type the following word: ")
print("fascinated interesting madly miniature flesh rain")
user_veri = getKeyStroke(register=False)

Enter your text: 
type the following word: 
fascinated interesting madly miniature flesh rain


In [9]:
eventList = pd.DataFrame(user_veri.eventList,columns=['subject','key','keyEvent','Time'])
strokeLen = len(eventList)
finalData = {}
veri_df = pd.DataFrame(columns=['subject','key','time'])
for j in range(0,strokeLen-1):
        finalData['subject'] = 'guest'
        finalData['key'] = chr(eventList.iloc[j].key) + " " + chr(eventList.iloc[j+1].key)
        finalData['time'] = (int(eventList.iloc[j+1].Time) - int(eventList.iloc[j].Time))/1000
        veri_df = veri_df.append(finalData,ignore_index=True )
veri_df = veri_df.groupby(['key']).mean().reset_index()
veri_df.insert(0,'subject',['guest']*veri_df.shape[0])
veri_pivot = veri_df.pivot(index='subject', columns='key', values = 'time')

In [10]:
clf.predict(veri_pivot[veri_pivot.columns[1:]].to_numpy())

array(['tester'], dtype=object)